## Import libraries 

In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_absolute_error

plt.style.use('seaborn')
sns.set(font_scale=2.5)

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


## DATA Preprocessing 

In [126]:
class suicides(object):
    def __init__(self):
        self.df = suicides.df
        self.df_test = suicides.df_test
        self.df_all_feature_var_names = []
        self.df_test_all_feature_var_names = []
        
    _non_numerical_feature_names = []
    _numerical_feature_names = []
    _is_one_hot_encoder = 0
    _feature_names_num = []
    _is_not_import_data = 0
    
    
    df = pd.read_csv('C:/Users/abrar/Desktop/2019/Suicide_Prediction-master/Suicide_Prediction-master/train_r.csv',sep=';')
    df_test = pd.read_csv('C:/Users/abrar/Desktop/2019/Suicide_Prediction-master/Suicide_Prediction-master/test_r.csv',sep=';')
    
    
    # extract numeric features 
    def extract_numerical_features(df):
        '''First Function Returns Numeric Features.'''
        df = df.copy()
        numerical_features = pd.Series(data=False, index=df.columns, dtype=bool)
        
        for feature in df.columns:
            if any(tuple(df[feature].map(lambda x: type(x)) == int)) or \
                any(tuple(df[feature].map(lambda x: type(x)) == float)) & \
                    (not any(tuple(df[feature].map(lambda x: type(x)) == str))):
                numerical_features[feature] = 1
    
        return numerical_features[numerical_features == 1].index
    print (extract_numerical_features.__doc__)
    # extract non numeric featurses 
    def extract_non_numerical_features(df):
        '''Second  Function Returns non_numerical_features.'''
       
        df = df.copy()
    
        return df.select_dtypes(exclude=[np.number])
    print (extract_non_numerical_features.__doc__)
    
    # encode the categorical features 
    def encode_labels_in_numeric_format(df, estimated_var):
        '''Third  Function Returns numeric features from categories.'''
         
        feature_names_num = ''.join([estimated_var, '_count'])
        mask = ~df[estimated_var].isnull()
        df[feature_names_num] = df[estimated_var]
        df.loc[mask, tuple([feature_names_num])] = df[estimated_var].factorize()[0][mask[mask==1].index]
    print (encode_labels_in_numeric_format.__doc__)
    # insert the lables into the lables after encoding 
    def label_classes(df, estimated_var):
        '''Fourth Function Returns Classes_features.'''
        
        le = LabelEncoder()
        le.fit(df[estimated_var].values)
        
        return le.classes_
    print (label_classes.__doc__)
    
    #  one hot encoding
    def one_hot_encoder(df, estimated_var):
        """Summary or Description of the Function 5 to encode categorical_feature = ['sex','age','region']

    Parameters:
    argument1 (int): dataframes , categorical feature 

    Returns:
    changes data frames  format to endoding

   """
       
        df_class = df.copy()
        ohe = OneHotEncoder()
        label_classes = df_class[estimated_var].factorize()[1]
        new_one_hot_encoded_features = [''.join([estimated_var, '_', x]) for x in label_classes]
        mask = ~df[estimated_var].isnull()
        feature_var_values = ohe.fit_transform(np.reshape(np.array(df[''.join([estimated_var, '_count'])][mask].values), 
                                                      (df[mask].shape[0], 1))).toarray().astype(int)

        for ite in new_one_hot_encoded_features:
            df[ite] = df[estimated_var]
        df.loc[mask, tuple(new_one_hot_encoded_features)] = feature_var_values
    print (one_hot_encoder.__doc__)
    
    


First Function Returns Numeric Features.
Second  Function Returns non_numerical_features.
Third  Function Returns numeric features from categories.
Fourth Function Returns Classes_features.
Summary or Description of the Function 5 to encode categorical_feature = ['sex','age','region']

    Parameters:
    argument1 (int): dataframes , categorical feature 

    Returns:
    changes data frames  format to endoding

   


In [127]:
train = suicides.df
test = suicides.df_test

In [128]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26916 entries, 0 to 26915
Data columns (total 16 columns):
country               26916 non-null object
year                  26916 non-null int64
sex                   26916 non-null object
age                   26916 non-null object
suicides_no           26916 non-null int64
population            26916 non-null int64
suicides_per100k      26916 non-null object
country-year          26916 non-null object
HDI_for_year          26916 non-null object
gdp_for_year          26916 non-null object
gdp_per_capita        26916 non-null int64
generation            26916 non-null object
country_code          26916 non-null object
country-code          26916 non-null object
region                26916 non-null object
unemployment index    26916 non-null object
dtypes: int64(4), object(12)
memory usage: 3.3+ MB


In [129]:
def str_float(value):
    '''  Function str_float Returns format , to '.'.'''
    value2=value.replace(',', '.')
    value2=float(value2)
    return value2
# FUNCTION TO UPDATE THE FLOAT FEATURE
train['gdp_for_year']=train['gdp_for_year'].apply(str_float)
test['gdp_for_year']=test['gdp_for_year'].apply(str_float)
print (str_float.__doc__)

def str_comma_removal_to_float(value):
    '''  Function str_float Returns format , to ' '.'''
    value=value.replace(',','')
    value=float(value)
    return value
train['unemployment index']=train['unemployment index'].apply(str_comma_removal_to_float)
test['unemployment index']=test['unemployment index'].apply(str_comma_removal_to_float)
print (str_comma_removal_to_float.__doc__)

  Function str_float Returns format , to '.'.
  Function str_float Returns format , to ' '.


In [130]:
#add region 
Europe = ["Albania","Russian Federation","France","Ukraine","Germany","Poland","United Kingdom",
         "Italy","Spain","Hungary","Romania","Belgium","Belarus","Netherlands","Austria",
         "Czech Republic","Sweden","Bulgaria","Finland","Lithuania","Switzerland","Serbia",
         "Portugal","Croatia","Norway","Denmark","Slovakia","Latvia","Greece","Slovenia",
         "Turkey","Estonia","Georgia","Albania","Luxembourg","Armenia","Iceland","Montenegro",
         "Cyprus","Bosnia and Herzegovina","San Marino","Malta","Ireland"]
NorthAmerica = ["United States","Mexico","Canada","Cuba","El Salvador","Puerto Rico",
                "Guatemala","Costa Rica","Nicaragua","Belize","Jamaica"]
SouthAmerica = ["Brazil","Colombia", "Chile","Ecuador","Uruguay","Paraguay","Argentina",
                "Panama","Guyana","Suriname"]
MiddleEast = ["Kazakhstan","Uzbekistan","Kyrgyzstan","Israel","Turkmenistan","Azerbaijan",
              "Kuwait","United Arab Emirates","Qatar","Bahrain","Oman"]
Asia = ["Japan","Republic of Korea", "Thailand", "Sri Lanka","Philippines","New Zealand",
        "Australia","Singapore","Macau","Mongolia"]

def match_region(x):
    if x in Europe:
        x = 'Europe'
    elif x in NorthAmerica:
        x = 'North America'
    elif x in SouthAmerica:
        x = 'South America'
    elif x in MiddleEast:
        x = 'MiddleEast'
    elif x in Asia:
        x = 'Asia'
    else:
        x = 'Island Nation'
    return x
  
  
train['region'] = train.country.map(lambda x: match_region(x))
  
test['region'] = test.country.map(lambda x: match_region(x))

In [131]:
categorical_feature = ['sex','age','region']


In [132]:
suicides.encode_labels_in_numeric_format(train, categorical_feature[0])
suicides.encode_labels_in_numeric_format(train, categorical_feature[1])
suicides.encode_labels_in_numeric_format(train, categorical_feature[2])
#suicides.encode_labels_in_numeric_format(train, categorical_feature[3])

suicides.encode_labels_in_numeric_format(test, categorical_feature[0])
suicides.encode_labels_in_numeric_format(test, categorical_feature[1])
suicides.encode_labels_in_numeric_format(test, categorical_feature[2])
#suicides.encode_labels_in_numeric_format(test, categorical_feature[3])

train.head()

,country,year,sex,age,suicides_no,population,suicides_per100k,country-year,HDI_for_year,gdp_for_year,gdp_per_capita,generation,country_code,country-code,region,unemployment index,sex_count,age_count,region_count
0,Albania,1987,male,15-24 years,21,312900,"6,71",Albania1987,"0,673",2.156625e+09,796,Generation X,AL,ALB,Europe,6.100000e+09,0,0,0
1,Albania,1987,male,35-54 years,16,308000,"5,19",Albania1987,"0,673",2.156625e+09,796,Silent,AL,ALB,Europe,6.100000e+09,0,1,0
2,Albania,1987,female,15-24 years,14,289700,"4,83",Albania1987,"0,673",2.156625e+09,796,Generation X,AL,ALB,Europe,6.100000e+09,1,0,0
3,Albania,1987,male,75+ years,1,21800,"4,59",Albania1987,"0,673",2.156625e+09,796,G.I. Generation,AL,ALB,Europe,6.100000e+09,0,2,0
4,Albania,1987,male,25-34 years,9,274300,"3,28",Albania1987,"0,673",2.156625e+09,796,Boomers,AL,ALB,Europe,6.100000e+09,0,3,0


In [133]:
suicides.one_hot_encoder(train, categorical_feature[0])
suicides.one_hot_encoder(train, categorical_feature[1])
suicides.one_hot_encoder(train, categorical_feature[2])
#suicides.one_hot_encoder(train, categorical_feature[3])



suicides.one_hot_encoder(test, categorical_feature[0])
suicides.one_hot_encoder(test, categorical_feature[1])
suicides.one_hot_encoder(test, categorical_feature[2])
#suicides.one_hot_encoder(test, categorical_feature[3])
train.head()

,country,year,sex,age,suicides_no,population,suicides_per100k,country-year,HDI_for_year,gdp_for_year,...,age_75+ years,age_25-34 years,age_55-74 years,age_5-14 years,region_Europe,region_Island Nation,region_South America,region_Asia,region_MiddleEast,region_North America
0,Albania,1987,male,15-24 years,21,312900,"6,71",Albania1987,"0,673",2.156625e+09,...,0,0,0,0,1,0,0,0,0,0
1,Albania,1987,male,35-54 years,16,308000,"5,19",Albania1987,"0,673",2.156625e+09,...,0,0,0,0,1,0,0,0,0,0
2,Albania,1987,female,15-24 years,14,289700,"4,83",Albania1987,"0,673",2.156625e+09,...,0,0,0,0,1,0,0,0,0,0
3,Albania,1987,male,75+ years,1,21800,"4,59",Albania1987,"0,673",2.156625e+09,...,1,0,0,0,1,0,0,0,0,0
4,Albania,1987,male,25-34 years,9,274300,"3,28",Albania1987,"0,673",2.156625e+09,...,0,1,0,0,1,0,0,0,0,0


In [134]:
train.columns

Index(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides_per100k', 'country-year', 'HDI_for_year', 'gdp_for_year',
       'gdp_per_capita', 'generation', 'country_code', 'country-code',
       'region', 'unemployment index', 'sex_count', 'age_count',
       'region_count', 'sex_male', 'sex_female', 'age_15-24 years',
       'age_35-54 years', 'age_75+ years', 'age_25-34 years',
       'age_55-74 years', 'age_5-14 years', 'region_Europe',
       'region_Island Nation', 'region_South America', 'region_Asia',
       'region_MiddleEast', 'region_North America'],
      dtype='object')

In [135]:
train = train.drop(categorical_feature[0], axis=1)
train = train.drop(categorical_feature[1], axis=1)
train = train.drop(categorical_feature[2], axis=1)
#train = train.drop(categorical_feature[3], axis=1)
train = train.drop('country-year', axis=1)


test = test.drop(categorical_feature[0], axis=1)
test = test.drop(categorical_feature[1], axis=1)
test = test.drop(categorical_feature[2], axis=1)
#test = test.drop(categorical_feature[3], axis=1)
test = test.drop('country-year', axis=1)


train.head()

,country,year,suicides_no,population,suicides_per100k,HDI_for_year,gdp_for_year,gdp_per_capita,generation,country_code,...,age_75+ years,age_25-34 years,age_55-74 years,age_5-14 years,region_Europe,region_Island Nation,region_South America,region_Asia,region_MiddleEast,region_North America
0,Albania,1987,21,312900,"6,71","0,673",2.156625e+09,796,Generation X,AL,...,0,0,0,0,1,0,0,0,0,0
1,Albania,1987,16,308000,"5,19","0,673",2.156625e+09,796,Silent,AL,...,0,0,0,0,1,0,0,0,0,0
2,Albania,1987,14,289700,"4,83","0,673",2.156625e+09,796,Generation X,AL,...,0,0,0,0,1,0,0,0,0,0
3,Albania,1987,1,21800,"4,59","0,673",2.156625e+09,796,G.I. Generation,AL,...,1,0,0,0,1,0,0,0,0,0
4,Albania,1987,9,274300,"3,28","0,673",2.156625e+09,796,Boomers,AL,...,0,1,0,0,1,0,0,0,0,0


In [136]:
train = train[train['year']> 2008]
train["HDI_for_year"] = train["HDI_for_year"].str.replace(",","").astype(float)

In [137]:
test.info()
test["HDI_for_year"] = test["HDI_for_year"].str.replace(",","").astype(float)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904 entries, 0 to 903
Data columns (total 29 columns):
country                 904 non-null object
year                    904 non-null int64
suicides_no             904 non-null int64
population              904 non-null int64
suicides_per100k        904 non-null object
HDI_for_year            904 non-null object
gdp_for_year            904 non-null float64
gdp_per_capita          904 non-null int64
generation              904 non-null object
country_code            904 non-null object
country-code            904 non-null object
unemployment index      904 non-null float64
sex_count               904 non-null int64
age_count               904 non-null int64
region_count            904 non-null int64
sex_female              904 non-null int32
sex_male                904 non-null int32
age_55-74 years         904 non-null int32
age_15-24 years         904 non-null int32
age_25-34 years         904 non-null int32
age_35-54 years         9

## PreProcessing ( Feature Engineering )

In [138]:


test = test.drop(['suicides_per100k'], axis=1)
train = train.drop(['suicides_per100k'], axis=1)


train['suicides_100k_pop'] = train['suicides_no'] / train['population'] * 100000
 

features_drop_train = ['suicides_no','sex_count','generation','age_count','region_count']
features_drop_test = ['sex_count','age_count','generation','region_count']

test = test.drop(features_drop_test, axis=1)
train = train.drop(features_drop_train, axis=1)


In [139]:
con = []
for i in test.country.unique():
    con.append(train[train.country == i])

In [140]:
train = pd.concat(con, axis=0)

In [141]:
train.head()

,country,year,population,HDI_for_year,gdp_for_year,gdp_per_capita,country_code,country-code,unemployment index,sex_male,...,age_25-34 years,age_55-74 years,age_5-14 years,region_Europe,region_Island Nation,region_South America,region_Asia,region_MiddleEast,region_North America,suicides_100k_pop
528,Antigua and Barbuda,2009,8199,781666667.0,1.224253e+09,14335,AG,ATG,6.0,0,...,0,0,0,0,1,0,0,0,0,0.0
529,Antigua and Barbuda,2009,6917,781666667.0,1.224253e+09,14335,AG,ATG,6.0,0,...,1,0,0,0,1,0,0,0,0,0.0
530,Antigua and Barbuda,2009,14629,781666667.0,1.224253e+09,14335,AG,ATG,6.0,0,...,0,0,0,0,1,0,0,0,0,0.0
531,Antigua and Barbuda,2009,8514,781666667.0,1.224253e+09,14335,AG,ATG,6.0,0,...,0,0,1,0,1,0,0,0,0,0.0
532,Antigua and Barbuda,2009,4793,781666667.0,1.224253e+09,14335,AG,ATG,6.0,0,...,0,1,0,0,1,0,0,0,0,0.0


In [142]:
test.head()

,country,year,suicides_no,population,HDI_for_year,gdp_for_year,gdp_per_capita,country_code,country-code,unemployment index,...,age_25-34 years,age_35-54 years,age_5-14 years,age_75+ years,region_Island Nation,region_South America,region_Europe,region_Asia,region_North America,region_MiddleEast
0,Antigua and Barbuda,2015,1,6403,781666667.0,1.364863e+09,14853,AG,ATG,6.0,...,0,0,0,0,1,0,0,0,0,0
1,Antigua and Barbuda,2015,0,8561,781666667.0,1.364863e+09,14853,AG,ATG,6.0,...,0,0,0,0,1,0,0,0,0,0
2,Antigua and Barbuda,2015,0,7740,781666667.0,1.364863e+09,14853,AG,ATG,6.0,...,1,0,0,0,1,0,0,0,0,0
3,Antigua and Barbuda,2015,0,15323,781666667.0,1.364863e+09,14853,AG,ATG,6.0,...,0,1,0,0,1,0,0,0,0,0
4,Antigua and Barbuda,2015,0,8239,781666667.0,1.364863e+09,14853,AG,ATG,6.0,...,0,0,1,0,1,0,0,0,0,0


In [143]:
train[train.country == 'Antigua and Barbuda'] = train[train.country == 'Antigua and Barbuda'].fillna(0)
test[test.country == 'Antigua and Barbuda'] = test[test.country == 'Antigua and Barbuda'].fillna(0)

train = train.fillna(method='ffill')
test = test.fillna(method='ffill')

In [144]:
train.isna().sum()

country                 0
year                    0
population              0
HDI_for_year            0
gdp_for_year            0
gdp_per_capita          0
country_code            0
country-code            0
unemployment index      0
sex_male                0
sex_female              0
age_15-24 years         0
age_35-54 years         0
age_75+ years           0
age_25-34 years         0
age_55-74 years         0
age_5-14 years          0
region_Europe           0
region_Island Nation    0
region_South America    0
region_Asia             0
region_MiddleEast       0
region_North America    0
suicides_100k_pop       0
dtype: int64

In [145]:
test.isna().sum()

country                 0
year                    0
suicides_no             0
population              0
HDI_for_year            0
gdp_for_year            0
gdp_per_capita          0
country_code            0
country-code            0
unemployment index      0
sex_female              0
sex_male                0
age_55-74 years         0
age_15-24 years         0
age_25-34 years         0
age_35-54 years         0
age_5-14 years          0
age_75+ years           0
region_Island Nation    0
region_South America    0
region_Europe           0
region_Asia             0
region_North America    0
region_MiddleEast       0
dtype: int64

In [146]:
train_test_data=[train,test]
def normalize(X):
    return np.abs((X-np.mean(X))/np.std(X))
for dataset in train_test_data:
    dataset['gdp_per_capita']=normalize(dataset['gdp_per_capita'])
    dataset['gdp_for_year']=normalize(dataset['gdp_for_year'])
test.head()

,country,year,suicides_no,population,HDI_for_year,gdp_for_year,gdp_per_capita,country_code,country-code,unemployment index,...,age_25-34 years,age_35-54 years,age_5-14 years,age_75+ years,region_Island Nation,region_South America,region_Europe,region_Asia,region_North America,region_MiddleEast
0,Antigua and Barbuda,2015,1,6403,781666667.0,0.310791,0.479044,AG,ATG,6.0,...,0,0,0,0,1,0,0,0,0,0
1,Antigua and Barbuda,2015,0,8561,781666667.0,0.310791,0.479044,AG,ATG,6.0,...,0,0,0,0,1,0,0,0,0,0
2,Antigua and Barbuda,2015,0,7740,781666667.0,0.310791,0.479044,AG,ATG,6.0,...,1,0,0,0,1,0,0,0,0,0
3,Antigua and Barbuda,2015,0,15323,781666667.0,0.310791,0.479044,AG,ATG,6.0,...,0,1,0,0,1,0,0,0,0,0
4,Antigua and Barbuda,2015,0,8239,781666667.0,0.310791,0.479044,AG,ATG,6.0,...,0,0,1,0,1,0,0,0,0,0


In [147]:
# train = pd.read_csv('./train.csv')
# test = pd.read_csv('./test1.csv')

##  Modeling


In [148]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet, ElasticNetCV
from pandas import Series, DataFrame
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor  


df_predict = DataFrame()
 
for i in train['country'].unique():
  
  train_by_country = train[train['country']==i]
  
  y = train_by_country['suicides_100k_pop']
  X = train_by_country.drop(['suicides_100k_pop','country','population','country_code','country-code'],axis=1) #independent columns
  #print(y)

   
  X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .01, random_state = 100)


  if (i== "United States"  or i== "Republic of Korea" or i== "Japan" or i== "Russian Federation" or i == 'Poland' or i == 'Kazakhstan'): 
    model = XGBRegressor(learning_rate=0.4, reg_lambda=.5, subsample=.9).fit(X_train, y_train)
    print(model.feature_importances_)
    
  elif (i == "Ukraine" or i == "Hungary" or i == "Brazil" or i == "Thailand" or i == "Mexico" or i == "Ecuador" or 
        i == "Colombia" or i == "Germany" or i == "Argentina" or i == "Israel" or i == "Italy" or i == "Australia" or
        i == "Czech Republic" or i == "Romania"):
    model = XGBRegressor(learning_rate=0.4, reg_lambda=.4, subsample=.9).fit(X_train, y_train)
    print(model.feature_importances_)
  else:
    #model  = GradientBoostingRegressor(learning_rate=0.5).fit(X_train, y_train)
    #model = XGBRegressor(learning_rate=0.4, reg_lambda=.5, subsample=.7).fit(X_train, y_train)
    model = XGBRegressor(learning_rate=0.4, reg_lambda=.5, subsample=.9).fit(X_train, y_train)
    #print(model.feature_importances_)
    
    
  test_by_country = test[test['country']==i]

  test_by_country['generation_G.I. Generation']=0
  


  test_by_country_2= test_by_country.drop(['population','country'],axis=1)
  test_by_country_2 = test_by_country_2.reindex(columns = X.columns)


  ss= pd.Series(model.predict(test_by_country_2)).to_frame('predict')



  test_by_country=test_by_country.reset_index()
  test_result =pd.concat([test_by_country , pd.Series(model.predict(test_by_country_2)).to_frame('predict')] , axis=1)

  test_result['predicted_suicides_no']= test_result['predict'] /100000 * test_result['population']
  test_result= test_result.loc[:, ['country', 'year','predict','population','predicted_suicides_no']]

  df_predict=df_predict.append(test_result)

df_predict




[0.00332588 0.00330694 0.00367853 0.         0.         0.27972028
 0.         0.04556101 0.0626768  0.11048765 0.00175775 0.0032562
 0.48622906 0.         0.         0.         0.         0.
 0.        ]
[0.00354804 0.00186006 0.00220935 0.         0.00160007 0.29723144
 0.         0.03677973 0.02976795 0.03367145 0.00106733 0.02713036
 0.56513417 0.         0.         0.         0.         0.
 0.        ]
[3.78188619e-04 1.29398028e-03 1.20757344e-04 1.23252161e-04
 0.00000000e+00 2.63823956e-01 0.00000000e+00 6.52315468e-02
 2.22194986e-03 1.78636998e-01 7.60743060e-05 4.35392698e-03
 4.83739287e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
[0.00305573 0.00138667 0.00246113 0.         0.00372521 0.32424042
 0.         0.03785213 0.03461634 0.05118016 0.00426188 0.00075033
 0.53647006 0.         0.         0.         0.         0.
 0.        ]
[0.00144982 0.00180377 0.00510037 0.00841192 0.00048027 0.23535189
 0.         0.0608028  0.

,country,year,predict,population,predicted_suicides_no
0,Antigua and Barbuda,2015,0.000000,6403,0.000000
1,Antigua and Barbuda,2015,0.000000,8561,0.000000
2,Antigua and Barbuda,2015,0.000000,7740,0.000000
3,Antigua and Barbuda,2015,0.000000,15323,0.000000
4,Antigua and Barbuda,2015,0.000000,8239,0.000000
5,Antigua and Barbuda,2015,0.000000,1724,0.000000
6,Antigua and Barbuda,2015,0.000000,8331,0.000000
7,Antigua and Barbuda,2015,0.000000,7059,0.000000
8,Antigua and Barbuda,2015,0.000000,13370,0.000000
9,Antigua and Barbuda,2015,0.000000,8268,0.000000


In [149]:
X.head()

,year,HDI_for_year,gdp_for_year,gdp_per_capita,unemployment index,sex_male,sex_female,age_15-24 years,age_35-54 years,age_75+ years,age_25-34 years,age_55-74 years,age_5-14 years,region_Europe,region_Island Nation,region_South America,region_Asia,region_MiddleEast,region_North America
26592,2009,746555556.0,0.353114,0.673218,7.740100e+09,1,0,0,0,1,0,0,0,0,0,1,0,0,0
26593,2009,746555556.0,0.353114,0.673218,7.740100e+09,1,0,0,0,0,0,1,0,0,0,1,0,0,0
26594,2009,746555556.0,0.353114,0.673218,7.740100e+09,1,0,0,0,0,1,0,0,0,0,1,0,0,0
26595,2009,746555556.0,0.353114,0.673218,7.740100e+09,1,0,0,1,0,0,0,0,0,0,1,0,0,0
26596,2009,746555556.0,0.353114,0.673218,7.740100e+09,1,0,1,0,0,0,0,0,0,0,1,0,0,0


## Test

In [150]:
merge_data1 = df_predict['predicted_suicides_no'].groupby(df_predict['country']).sum()
merge_data2 =pd.DataFrame({'country':merge_data1.index, 'predicted_suicides_no':merge_data1.values})

merge_data2.to_csv('result_366.csv')

answer= pd.read_csv('C:/Users/abrar/Desktop/2019/Suicide_Prediction-master/Suicide_Prediction-master/2015_suicide_no.csv')
answer.columns = ['country', 'suicides_no']

merge_answer= pd.merge(merge_data2, answer , on ='country', how='left')
merge_answer['error']=merge_answer['suicides_no']-merge_answer['predicted_suicides_no']
merge_answer['error_Abs']=abs(merge_answer['error'])
merge_answer1=merge_answer.sort_values('error_Abs',ascending=False)
merge_answer1.to_csv('after_merge.csv')

In [151]:
true_value = merge_answer1.suicides_no
prediction_value = merge_answer1.predicted_suicides_no

print('RMSE = ', np.sqrt(mean_squared_error(prediction_value, true_value)))
print('MAE =', mean_absolute_error(prediction_value,true_value))
print('MAE =', np.log(mean_absolute_error(prediction_value,true_value)))



RMSE =  815.4110906755627
MAE = 403.76597311604024
MAE = 6.000835435651628
